## 1. Setup StockX

In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
util.startLoop()
import stock

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=15)
sx  = stock.StockX(ib, 'SABR')
spy = stock.StockX(ib, 'SPY')

In [2]:
spy.RUN_DAILY(isMarket=True)

sx.set_ls('LONG')
sx.RUN_DAILY(spy)

-------------------------------------------------------------------------------------------------
get_hist_data            : SPY 1 day (file_interval='1_day')
Data Path                : C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store
Data Stored    - Start   : 2023-12-07 00:00:00, End: 2025-03-18 00:00:00, Rows: 319
Data Requested - Start   : 2024-03-18 00:00:01, End: now
Missing dates            : 1
Getting missing data     : [('2025-03-17', '2025-03-18')]
-------------------------------------------------------------------------------------------------
StockX::setup_frame: SPY 1 day ohlcv 52 weeksAgo now force_download=False : (250, 5)
-------------------------------------------------------------------------------------------------
get_hist_data            : SPY 1 hour (file_interval='1_hour')
Data Path                : C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store
Data Stored    - Start   : 2024-02-16 04:00:00, End: 2025-03-18 12:00:00,

In [3]:
sx.get_frame_data('1 hour').columns

Index(['open', 'high', 'low', 'close', 'volume', 'SPY_low', 'SPY_high',
       'SPY_open', 'SPY_volume', 'SPY_close', 'MA_cl_50', 'MA_cl_150',
       'MA_cl_200', 'ATR_14', 'HP_hi_3', 'LP_lo_3', 'HP_hi_10', 'LP_lo_10',
       'Res_1', 'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper',
       'Res_2_Lower', 'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2',
       'Sup_2_Upper', 'Sup_2_Lower', 'VWAP_session', 'VOL_Acum',
       'pre_mkt_high', 'pre_mkt_low', 'intraday_high_9.35',
       'intraday_low_9.35', 'intraday_high', 'intraday_low', 'prev_day_high',
       'prev_day_low', '1 day_Res_1_Lower', '1 day_Sup_1_Upper',
       '1 day_MA_cl_50', '1 day_MA_cl_200', 'VOL_TODC_10'],
      dtype='object')

In [4]:
sx.set_daily_stats(printStats=True)

Using cached data for SABR (age: 2025-03-17 20:28:24)


,symbol,snapshotTime,status,price,marketCap,currentPrice,peRatio,vol10dayAvg,vol_TODC,priceChgPct,breaksAbove10DayAvg,breaksAbove50MA,breaksAbove200MA,breaksBelow50MA,breaksBelow200MA,scoreVol,scoreGaps,scoreRTM,score_1D,validAv_1D
0,SABR,2025-03-18 19:03:27,DAILY_TA,3.515,1285.612,3.33,-99999.99,5.31387,0.0,2.0,False,False,False,False,False,4.97,20.0,25.9,16.96,14.3


In [5]:
sx.get_frame_data('1 day', colsContains='RTM').tail(10)

,RTM_L_Res_1_Lower,VAL_L_RTM_L_Res_1_Lower_>_50,Score_LONG_RTM
date,,,
2025-03-05,12.043011,0.0,12.0
2025-03-06,9.853372,0.0,9.9
2025-03-07,16.800000,0.0,16.8
2025-03-10,15.757085,0.0,15.8
2025-03-11,31.973958,0.0,32.0
2025-03-12,2.441860,0.0,2.4
2025-03-13,0.890302,0.0,0.9
2025-03-14,42.091803,0.0,42.1
2025-03-17,32.151565,0.0,32.2


In [6]:
sx.get_frame('1 day').plot()

In [ ]:
vol_pct_chg = sx.get_frame_data('1 hour')['VOL_TODC_10'].iat[-1]


print(f'Volume percent change: {vol_pct_chg:.2f}%')

## 2. Fundamentals

In [ ]:
sx.RUN_FUNDAMENTALS(allowedETFs=['XLY', 'XLK', 'XLC'], maxDaysOld=10)

## 3. Setup

In [ ]:
sx.RUN_SETUP(spy, dataType='ohlcv', forceDownload=False, endDate='2025-02-20')

## 4. PreMarket

In [ ]:
sx.RUN_PRE_MARKET('LONG', timeToRun='09:00', minPreMarketScore=50, displayCharts=True, printStats=True) # runs daily and premarket volume analysis

In [ ]:
sx.display_columns('1 day')
sx.display_columns('1 day', contains='SPY')

## 5. Intraday

In [ ]:
sx.RUN_INTRADAY(updateEverySeconds=60, myTradingTimes=[('9:35', '15:00')], maxTrades=5, displayCharts=True, logTrades=True)

In [ ]:
import pandas as pd
import strategies.preset_strats as ps
import strategies.signals as sig
import strategies.ta as ta
from typing import Dict, Any, List
from chart.chart import ChartArgs

sx.import_all_market_data(spy)
sx.set_ls('LONG')
sx.setup_TA_1D(lookBack=25, atrSpan=14, sigRow=3, validationRow=4)

if sx.day_score_passed(minScore=50):
    sx.setup_TA_intraday(lookBack=200, atrSpan=50, sigRow=3, validationRow=4)
    
    if sx.pre_market_passed(minScore=50):
        sx.RUN(updateEvery=30, myTradingTimes=[('9:35', '15:00')], maxTrades=5, displayCharts=False, logTrades=True)




spy.frames['1 day'].plot()
sx.frames['1 day'].plot()
sx.frames['1 hour'].plot()
sx.frames['5 mins'].plot()

In [ ]:
sx.display_columns('5 mins', contains='PB')

In [ ]:
dfx = sx.display_columns('5 mins', contains='PB')
dfx.loc['2025-02-20 09:35:00':].head(50)

 # StockX Score

In [ ]:
ib.disconnect()

In [ ]:
from dataclasses import dataclass

